In [2]:
!pip install boto3 sagemaker comet_ml torch torchvision ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 726.7/726.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 33.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 43.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 37.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 57.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 55.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 54.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 58.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Install Libraries

In [3]:
import boto3
import os
import random
import shutil
from pathlib import Path
from datetime import datetime
from ultralytics import YOLO
import comet_ml

Matplotlib is building the font cache; this may take a moment.


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/ec2-user/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Connect to S3

In [4]:
# Setup boto3 clients
s3 = boto3.client('s3')
ssm = boto3.client('ssm')

In [5]:
# Get parameters from SSM
def get_parameters():
    response = ssm.get_parameters(
        Names=[
            '/edge-ai/bucket-name',
            '/edge-ai/comet-ml-api-key'
        ],
        WithDecryption=True
    )
    return {param['Name'].split('/')[-1]: param['Value'] for param in response['Parameters']}

In [6]:
params = get_parameters()
BUCKET_NAME = params['bucket-name']
COMET_ML_API_KEY = params['comet-ml-api-key']

In [7]:
# S3 paths
s3_img_prefix = 'training_data/new_data/images/'
s3_lbl_prefix = 'training_data/new_data/txt_files/'

# Local directories
base_dir = Path('/home/ec2-user/SageMaker/tmp/datasets')
train_img_dir = base_dir / 'train/images'
train_lbl_dir = base_dir / 'train/labels'
val_img_dir = base_dir / 'val/images'
val_lbl_dir = base_dir / 'val/labels'

In [8]:
# Create folders
for path in [train_img_dir, train_lbl_dir, val_img_dir, val_lbl_dir]:
    path.mkdir(parents=True, exist_ok=True)

# Load New Image Data

In [9]:
# List images
img_objs = s3.list_objects_v2(Bucket=BUCKET_NAME, Prefix=s3_img_prefix).get('Contents', [])
img_keys = [obj['Key'] for obj in img_objs if obj['Key'].endswith(('.jpg', '.png'))]
print(f"Done number of images {len(img_keys)}")

Done number of images 12


# Preprocess Images

In [10]:
# Random split
random.shuffle(img_keys)
split_idx = int(len(img_keys) * 0.1)
val_keys = img_keys[:split_idx]
train_keys = img_keys[split_idx:]
print(val_keys)
print(train_keys)

['training_data/new_data/images/6.png']
['training_data/new_data/images/10.png', 'training_data/new_data/images/15.png', 'training_data/new_data/images/16.png', 'training_data/new_data/images/13.png', 'training_data/new_data/images/17.png', 'training_data/new_data/images/14.png', 'training_data/new_data/images/8.png', 'training_data/new_data/images/3_789_20250415092609.jpg', 'training_data/new_data/images/5.png', 'training_data/new_data/images/4_123_20250415142226.jpg', 'training_data/new_data/images/2.png']


In [11]:
def download_and_place(keys, img_dest, lbl_dest):
    for key in keys:
        filename = os.path.basename(key)
        label_filename = filename.rsplit('.', 1)[0] + '.txt'
        label_key = s3_lbl_prefix + label_filename

        # Download image
        s3.download_file(BUCKET_NAME, key, str(img_dest / filename))
        
        # Download label if exists
        try:
            s3.download_file(BUCKET_NAME, label_key, str(lbl_dest / label_filename))
        except:
            print(f"Label file not found for {filename}, skipping.")

In [12]:
# Download images & labels
download_and_place(train_keys, train_img_dir, train_lbl_dir)
download_and_place(val_keys, val_img_dir, val_lbl_dir)

print("✅ Data prepared in /tmp/datasets/")

✅ Data prepared in /tmp/datasets/


# Load yaml file

In [13]:
yaml_key = 'training_data/new_data/data.yaml'

# Local path
local_yaml_path = Path('/home/ec2-user/SageMaker/tmp/datasets/data.yaml')
local_yaml_path.parent.mkdir(parents=True, exist_ok=True)

# Download data.yaml
s3.download_file(Bucket=BUCKET_NAME, Key=yaml_key, Filename=str(local_yaml_path))

print(f"✅ Downloaded data.yaml to {local_yaml_path}")

✅ Downloaded data.yaml to /home/ec2-user/SageMaker/tmp/datasets/data.yaml


# Load Latest Model

In [14]:
# List all model files and find the latest one
def get_latest_model_key(bucket_name):
    paginator = s3.get_paginator('list_objects_v2')
    result = paginator.paginate(Bucket=bucket_name, Prefix='models/')

    latest_key = None
    latest_time = datetime.min

    for page in result:
        for obj in page.get('Contents', []):
            key = obj['Key']
            if key.endswith('last.pt'):
                try:
                    parts = key.split('/')
                    date_str = f"{parts[1]}-{parts[2]}-{parts[3]}"
                    obj_date = datetime.strptime(date_str, '%Y-%m-%d')

                    if obj_date > latest_time:
                        latest_time = obj_date
                        latest_key = key
                except (IndexError, ValueError):
                    continue

    return latest_key

# Get the latest model key
latest_model_key = get_latest_model_key(BUCKET_NAME)

if latest_model_key:
    local_model_path = Path('./tmp/datasets/latest_model.pt')
    s3.download_file(BUCKET_NAME, latest_model_key, str(local_model_path))
    print(f"✅ Downloaded latest model to {local_model_path} from s3 location: {latest_model_key}")
else:
    print("❌ No model file found.")

✅ Downloaded latest model to tmp/datasets/latest_model.pt from s3 location: models/2025/04/18/last.pt


# Retrain Model

In [15]:
comet_ml_api_key = COMET_ML_API_KEY

In [16]:
# Set your Comet Api Key
!export COMET_API_KEY=comet_ml_api_key

In [17]:
comet_ml.login(project_name="IoT")

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)


Comet API key:  ········


COMET INFO: Valid Comet API Key saved in /home/ec2-user/.comet.config (set COMET_CONFIG to change where it is saved).


In [20]:
# Load a model
model = YOLO(local_model_path)  # load a pretrained model (recommended for training)

# Train the model
results = model.train(
    data=local_yaml_path,
    epochs=10,
    imgsz=640,
    batch=8,
    project="IoT",
    save_period=1,
    save_json=True,
)

Ultralytics 8.3.111 🚀 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 14918MiB)
engine/trainer: task=detect, mode=train, model=tmp/datasets/latest_model.pt, data=/home/ec2-user/SageMaker/tmp/datasets/data.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=1, cache=False, device=None, workers=8, project=IoT, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=True, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

COMET INFO: An experiment with the same configuration options is already running and will be reused.


Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4132.5±527.8 MB/s, size: 3729.1 KB)


train: Scanning /home/ec2-user/SageMaker/tmp/datasets/train/labels... 8 images, 3 backgrounds, 0 corrupt: 100%|██████████| 8/8 [00:00<00:00, 58.45it/s]

train: New cache created: /home/ec2-user/SageMaker/tmp/datasets/train/labels.cache


...

                   all          1          4      0.929          1      0.995      0.796



10 epochs completed in 0.010 hours.
Optimizer stripped from IoT/train3/weights/last.pt, 64.1MB
Optimizer stripped from IoT/train3/weights/best.pt, 64.1MB

Validating IoT/train3/weights/best.pt...
Ultralytics 8.3.111 🚀 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 14918MiB)
YOLOv10x summary (fused): 192 layers, 29,406,158 parameters, 0 gradients, 160.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 35.26it/s]


                   all          1          4      0.929          1      0.995      0.796
           Broken_Root          1          2      0.927          1      0.995      0.796
          Not_Free_Max          1          1          1          1      0.995      0.895
         Not_Free_Mand          1          1       0.86          1      0.995      0.697
Speed: 0.3ms preprocess, 21.2ms inference, 0.0ms loss, 0.3ms postprocess per image
Saving IoT/train3/predictions.json...
Results saved to IoT/train3


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : condemned_cactus_739
COMET INFO:     url                   : https://www.comet.com/ranxdug/iot/b318a6daef4e48f5a93599e3b9434be7
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [11]               : (0.0, 1.80285e-05)
COMET INFO:     lr/pg1 [11]               : (0.0, 1.80285e-05)
COMET INFO:     lr/pg2 [11]               : (0.0, 1.80285e-05)
COMET INFO:     metrics/mAP50(B)          : 0.995
COMET INFO:     metrics/mAP50-95(B) [11]  : (0.73029, 0.796)
COMET INFO:     metrics/precision(B) [11] : (0.74761, 0.95257)
COMET INFO:     metrics/recall(B)         : 1.0
COMET INFO:     model/GFLOPs              : 171.094
COMET INFO:     model/par

# Test Model Accuracy

In [21]:
# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

Ultralytics 8.3.111 🚀 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 14918MiB)
YOLOv10x summary (fused): 192 layers, 29,406,158 parameters, 0 gradients, 160.0 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1458.5±0.0 MB/s, size: 3264.3 KB)


val: Scanning /home/ec2-user/SageMaker/tmp/datasets/val/labels.cache... 1 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1/1 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  9.14it/s]


                   all          1          4      0.929          1      0.995      0.796
           Broken_Root          1          2      0.928          1      0.995      0.796
          Not_Free_Max          1          1          1          1      0.995      0.895
         Not_Free_Mand          1          1       0.86          1      0.995      0.697
Speed: 0.5ms preprocess, 89.6ms inference, 0.0ms loss, 0.3ms postprocess per image
Saving IoT/train32/predictions.json...
Results saved to IoT/train32


array([      0.796,       0.796,       0.796,       0.796,      0.8955,       0.796,       0.796,       0.796,      0.6965,       0.796])

# Save best.pt and last.pt to S3

In [23]:
# Define the local model path (update if you saved elsewhere)
MODEL_DIR = "/home/ec2-user/SageMaker/IoT/train3/weights"
BEST_MODEL = os.path.join(MODEL_DIR, "best.pt")
LAST_MODEL = os.path.join(MODEL_DIR, "last.pt")

# Create destination S3 path using current date
now = datetime.now()
s3_prefix = f"models/{now.year}/{now.month:02}/{now.day:02}"

# Upload files
def upload_model(file_path, file_name):
    s3_path = f"{s3_prefix}/{file_name}"
    s3.upload_file(file_path, BUCKET_NAME, s3_path)
    print(f"✅ Uploaded {file_name} to s3://{BUCKET_NAME}/{s3_path}")

upload_model(BEST_MODEL, "best.pt")
upload_model(LAST_MODEL, "last.pt")

✅ Uploaded best.pt to s3://edge-ai-s3/models/2025/04/19/best.pt
✅ Uploaded last.pt to s3://edge-ai-s3/models/2025/04/19/last.pt


In [28]:
def move_images_to_all_data():
    """ 
    Move all the images that have been trained to the training_data/all_data/yyyy/MM/dd
    """
    date_path = datetime.utcnow().strftime('%Y/%m/%d')
    destination_prefix = f"training_data/all_data/{date_path}/"

    folders_to_move = {
        'images': 'training_data/new_data/images/',
        'txt_files': 'training_data/new_data/txt_files/'
    }

    for file_type, prefix in folders_to_move.items():
        response = s3.list_objects_v2(Bucket=BUCKET_NAME, Prefix=prefix)

        if 'Contents' not in response:
            print(f"No files found in {prefix}")
            continue

        for obj in response['Contents']:
            source_key = obj['Key']
            if source_key.endswith('/'):
                continue  # skip folders

            file_name = source_key.split('/')[-1]
            destination_key = f"{destination_prefix}{file_name}"

            # Copy the object
            s3.copy_object(
                Bucket=BUCKET_NAME,
                CopySource={'Bucket': BUCKET_NAME, 'Key': source_key},
                Key=destination_key
            )

            # Delete the original object
            s3.delete_object(Bucket=BUCKET_NAME, Key=source_key)
            print(f"Moved: {source_key} -> {destination_key}")

In [29]:
move_images_to_all_data()

Moved: training_data/new_data/images/10.png -> training_data/all_data/2025/04/19/10.png
Moved: training_data/new_data/images/13.png -> training_data/all_data/2025/04/19/13.png
Moved: training_data/new_data/images/14.png -> training_data/all_data/2025/04/19/14.png
Moved: training_data/new_data/images/15.png -> training_data/all_data/2025/04/19/15.png
Moved: training_data/new_data/images/16.png -> training_data/all_data/2025/04/19/16.png
Moved: training_data/new_data/images/17.png -> training_data/all_data/2025/04/19/17.png
Moved: training_data/new_data/images/2.png -> training_data/all_data/2025/04/19/2.png
Moved: training_data/new_data/images/3_789_20250415092609.jpg -> training_data/all_data/2025/04/19/3_789_20250415092609.jpg
Moved: training_data/new_data/images/4_123_20250415142226.jpg -> training_data/all_data/2025/04/19/4_123_20250415142226.jpg
Moved: training_data/new_data/images/5.png -> training_data/all_data/2025/04/19/5.png
Moved: training_data/new_data/images/6.png -> traini

In [30]:
# Clean training runs
!rm -rf /home/ec2-user/SageMaker/IoT

# Clean YOLOv10 repo
!rm -rf /home/ec2-user/SageMaker/yolov10

# Clean temporary datasets
!rm -rf /home/ec2-user/SageMaker/tmp

# Clean SageMaker Trash completely

# (Optional) Remove checkpoints from Label Studio or others if you manually saved them elsewhere
# rm -rf /home/ec2-user/SageMaker/checkpoints_or_other_dirs_you_may_have
!rm -rf /home/ec2-user/SageMaker/.Trash-1000/files/yolov10
!rm -rf /home/ec2-user/SageMaker/.Trash-1000/files/*
!find /home/ec2-user/SageMaker -name "*.pt" -type f -delete

find: ‘/home/ec2-user/SageMaker/lost+found’: Permission denied


In [31]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
devtmpfs        7.7G     0  7.7G   0% /dev
tmpfs           7.7G  840K  7.7G   1% /dev/shm
tmpfs           7.7G  672K  7.7G   1% /run
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  135G   88G   48G  65% /
tmpfs           1.6G     0  1.6G   0% /run/user/0
/dev/nvme2n1     15G  268K   14G   1% /home/ec2-user/SageMaker
tmpfs           1.6G     0  1.6G   0% /run/user/1001
tmpfs           1.6G     0  1.6G   0% /run/user/1002
tmpfs           1.6G     0  1.6G   0% /run/user/1000


In [32]:
!du -h /home/ec2-user/SageMaker | sort -hr | head -20

du: cannot read directory ‘/home/ec2-user/SageMaker/lost+found’: Permission denied
264K	/home/ec2-user/SageMaker
84K	/home/ec2-user/SageMaker/.Trash-1000
76K	/home/ec2-user/SageMaker/.Trash-1000/info
64K	/home/ec2-user/SageMaker/.ipynb_checkpoints
16K	/home/ec2-user/SageMaker/lost+found
12K	/home/ec2-user/SageMaker/.virtual_documents
4.0K	/home/ec2-user/SageMaker/.Trash-1000/files
4.0K	/home/ec2-user/SageMaker/.sparkmagic
